# 远程推理应用示例3

本示例使用 Deeplabv3 模型远程推理，进行图像语义分割处理，以图片方式显示结果，简要说明了基于Atlas 200DK 开发板的远程推理应用开发方法之一。

## 主要特点：

1. 使用 Inferemote.DataSource 类获取输入，输入源可为图片目录、视频文件、摄像头或者图片的HTTP超链接地址；
2. 构造 Deeplabv3 模型算法对象，及其应用的回调函数；
3. 使用 Inferemote.testing 包，进行远程推理并对结果进行展示，展示方式可为图像窗口或WEB页面。

## 远程推理平台

本示例与所使用的远程推理集群服务器均运行在华为云ECS服务器上，集群中的12台Atlas200DK开发板部署于电子科技大学校园. 具体请参考 https://gitee.com/haojiash/atlas200dk_remote.

## 代码示例
【提醒】：注意修改 RemoteIP 和 测试输入 设置。

### 1. 导入软件包

In [1]:
"""
Inferemote: a Remote Inference Toolkit for Ascend 310

"""
import os, sys

from inferemote.testing.data_source import ImageFactory #ImageSource, VideoSource, UrlSource
from inferemote.testing import QuickTest, LivewebTest

### 2. 实例化语义分割应用算法的对象

In [2]:
from inferemote.airlab import Deeplabv3
air = Deeplabv3()

### 3. 语义分割应用的回调函数

In [3]:
''' 对语义分割结果进行进一步处理，将得到的mask合并在原图像中 '''
def test_func(image):
    result = air.inference_remote(image)
    image = air.make_mask(image, result)
    return image

### 4. 远程推理设置

In [4]:
''' 设置远程模型推理服务地址，以下为华为云ECS上的集群服务器： '''
remote = '192.168.1.123'

''' 关联远程推理端 '''
air.use_remote(remote)

03-18 17:28:03 [INFO]  -atlas- Trying model inference on REMOTE ``192.168.1.123''...


### 5. 设置输入图像源，可以为：摄像头、视频文件、图片目录或者图片的HTTP超链接地址 

In [5]:
# 以URL图片为输入
url_input = 'https://c7xcode.obs.cn-north-4.myhuaweicloud.com/models/deeplabv3/cat.jpg'

# 以图片目录为输入
dir_input = './data/pictures'

# 以摄像头为输入（如服务在ECS云服务器中，实际不具有摄像头）
camear_input = 'camera'

# 以视频文件为输入
video_input = './data/skiing.mp4'

# 设置用于本次测试的输入源
test_input = url_input

# 自动获取输入的Image流
ds = ImageFactory().get_stream(dir_input)

### 6. 运行测试

In [ ]:
''' 构造QuickTest对象，进行远程推理。
注意：1. QuickTest 在Jupyter中不能使用 opencv显示；2. LivewebTest 在jupyter中出现端口冲突无法使用 '''

test = QuickTest(ds, output='.', threads=4)

''' 开始测试 '''
test.run(test_func)

03-18 17:28:06 [WARNING]  --------- Data source CLOSED -----------



(Press CTRL+C to quit)



03-18 17:28:07 [INFO]  -atlas- Time costs: 0.012/0.658/0.007(s) and 0.677(s) for pre/remote/post and totally.
03-18 17:28:07 [INFO]  - job - fps: 1.34, 0.749s ellapsed for [ 1/1 ] frames.
03-18 17:28:07 [INFO]  - job - DONE: 0.751 seconds running for 1 frames toatally with fps: 1.332.
03-18 17:28:07 [INFO]  ======== Exited. ========


# Ends.